<a href="https://colab.research.google.com/github/hanakim120/visualization/blob/main/Pytorch_MNIST_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [1]:
!pip install wandb -qqq
import wandb

     |████████████████████████████████| 1.7 MB 4.0 MB/s 
     |████████████████████████████████| 181 kB 60.8 MB/s 
     |████████████████████████████████| 144 kB 68.7 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 


In [2]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## 데이터 준비

In [5]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



## Dataloader 생성
데이터셋을 순회 가능한 객체(iterable)로 감싸고, 자동화된 배치(batch), 샘플링(sampling), 섞기(shuffle) 및 다중 프로세스로 데이터 불러오기(multiprocess data loading)를 지원

In [6]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


## 모델

In [7]:
# 학습에 사용할 CPU나 GPU 장치를 얻습니다.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# 모델을 정의합니다. (nn.Module 을 상속받는 클래스(class)를 생성하여 정의)
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 모델 매개변수 최적화 (loss function, optimizer)

In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

## 훈련

- optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정합니다. 기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정합니다.
- loss.backwards()를 호출하여 예측 손실(prediction loss)을 역전파합니다. 
- PyTorch는 각 매개변수에 대한 손실의 변화도를 저장합니다.
- 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정합니다

In [15]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    #print(size, 'dataloade.dataset의 사이즈')
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        print(batch,'batch')
        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad() #Pytorch에서는 gradients값들을 추후에 backward를 해줄때 계속 더해주기 때문"에 우리는 항상 backpropagation을 하기전에 gradients를 zero로 만들어주고 시작을 해야함 (1iter 마다)
        loss.backward() # 역전파 해서 gradient가 저장
        optimizer.step() # weight를 업데이트 하는 시점, 모델 성능 개선

        wandb.log({"train_loss": loss})

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

## 테스트

In [16]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    #print(num_batches,'num_batches')
    model.eval() #  .eval()모드를 사용하면 모델 내부의 모든 layer가 evaluation 모드가 됩니다. evaluation 모드에서는 batchnorm, dropout과 같은 기능들이 사용되지 않습니다.
    test_loss, correct = 0, 0
    with torch.no_grad(): #autograd 엔진에게 이 정보를 알려주고 학습에서 제외. Backprop에 필요한 메모리 등을 절약할 수 있으므로 이 Tensor를 사용하여 계산 시 연산 속도가 빨라짐. 하지만 학습은 불가능
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    wandb.log({"test_loss": test_loss, "correct": correct})
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## 60000 dataloade.dataset의 사이즈
## 0 batch 부터 (batch size = 64) 100 단위로 loss 값 찍기 -> 6400 배치 마다 찍힘  
## num batch = len(dataloader) 157

In [17]:
wandb.init(
      # Set the project where this run will be logged
      project="hana-pytorch-practice", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"hana's experiment")
      # Track hyperparameters and run metadata)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")
wandb.finish()

Epoch 1
-------------------------------
0 batch
loss: 2.306589  [    0/60000]
1 batch
2 batch
3 batch
4 batch
5 batch
6 batch
7 batch
8 batch
9 batch
10 batch
11 batch
12 batch
13 batch
14 batch
15 batch
16 batch
17 batch
18 batch
19 batch
20 batch
21 batch
22 batch
23 batch
24 batch
25 batch
26 batch
27 batch
28 batch
29 batch
30 batch
31 batch
32 batch
33 batch
34 batch
35 batch
36 batch
37 batch
38 batch
39 batch
40 batch
41 batch
42 batch
43 batch
44 batch
45 batch
46 batch
47 batch
48 batch
49 batch
50 batch
51 batch
52 batch
53 batch
54 batch
55 batch
56 batch
57 batch
58 batch
59 batch
60 batch
61 batch
62 batch
63 batch
64 batch
65 batch
66 batch
67 batch
68 batch
69 batch
70 batch
71 batch
72 batch
73 batch
74 batch
75 batch
76 batch
77 batch
78 batch
79 batch
80 batch
81 batch
82 batch
83 batch
84 batch
85 batch
86 batch
87 batch
88 batch
89 batch
90 batch
91 batch
92 batch
93 batch
94 batch
95 batch
96 batch
97 batch
98 batch
99 batch
100 batch
loss: 2.284226  [ 6400/60000]


correct,▁▄▆██
test_loss,█▆▄▂▁
train_loss,████████▇▇▇▇▆▆▆▆▆▅▅▅▄▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁
correct,0.6478
test_loss,1.09233
train_loss,1.15502


## 모델 저장

In [12]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 모델 불러오기

In [13]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

## 예측

In [19]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Pullover", Actual: "Pullover"
